This kernel is a simple classification with augmentation examples (based on the Albumentations library)

A classifier can be used as part of the entire model :

 *  (as a pre-processing before doing the segmentation or as a post-processing as a helper to decide if there is no mask)

I have not tried this method yet, but you can try it.

You can also see the use of the Nvidia driver, It is using more efficiently the GPU memory. I think it also provides faster training 

The kernel is divided into two 
First part augmentation illustration for both images and masks 
Second part classification with augmentation 


Credits :  
  For the Triple Grand Master - Abhishek Thakur  
  ( https://www.kaggle.com/abhishek/mask-rcnn-with-augmentation-and-multiple-masks ) , As always learned a lot from him - the Data loader is coming from his kernels.
  Thanks for also loading and allowing the use of the data 
   https://www.kaggle.com/abhishek/siim-png-images
   
   Augmentation is based on this Github 
   https://github.com/albu/albumentations/blob/master/notebooks/example_kaggle_salt.ipynb
   

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, resnet34
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
from torchvision import transforms
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from matplotlib.pyplot import figure
from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.torch import ToTensor
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.metrics import confusion_matrix


import os
import cv2
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split
import collections
from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from PIL import Image, ImageFile
from apex import amp

## Albumentations

In [ ]:
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,    
    RandomGamma    
)

## Global Variables 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")

In [ ]:

SEED = 2098
EPOCH = 5



## Random Seed 

In [ ]:
def seed_all(seed= SEED):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_all(SEED)

## Load Meta Data 

In [ ]:
image_ids = pd.read_csv("../input/trainsiimacr/train_siimacr/train-rle.csv")
image_ids.head()

## Create Labels
Create Mask label  - 1 if Mask exists, 0 if not 

In [ ]:
image_ids['LenMask'] =  image_ids[' EncodedPixels'].apply(lambda x: len(x))
image_ids['Mask'] = np.where(image_ids['LenMask'] == 3 , 0,1)
image_ids.head()

## Split Meta Data 
Split Data to Train and Validation sets 

In [ ]:
train , val =  train_test_split(image_ids ,test_size=0.15, random_state=SEED ,stratify =image_ids.Mask)
val.shape

## Data loader 

In [ ]:
def get_image(path, id_code, size,Aug):
    img_path = path
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if Aug:
        image = RandCenterCrop(image)
        image = RandomBrightContrast(image)
    image = cv2.resize(image, (size, size))
    image = transforms.ToPILImage()(image)
    return image

## Rle2mask

In [ ]:
def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 1
        current_position += lengths[index]

    return mask.reshape(width, height)

Credit Here for This kernel (for the Data loading) - made by Abhishek Thakur
https://www.kaggle.com/abhishek/mask-rcnn-with-augmentation-and-multiple-masks

In [ ]:
class ImageDataLoader(torch.utils.data.Dataset):

    def __init__(self,train_file, image_dir , size =256,transform=None,Aug = False):
        
        self.df = train_file 
        self.image_dir = image_dir
        self.size = size
        self.name_frame = train_file['ImageId'].values
        self.label_frame = train_file['Mask'].values
        self.transform = transform
        self.Aug = Aug
        self.image_info = collections.defaultdict(dict)
        
        
        counter = 0
        for index, row in tqdm(self.df.iterrows(), total=len(self.df)):
            image_id = row['ImageId']
            image_path = os.path.join(self.image_dir, image_id)
            if os.path.exists(image_path + '.png') :
                self.image_info[counter]["image_id"] = image_id
                self.image_info[counter]["image_path"] = image_path
                self.image_info[counter]["annotations"] = row[" EncodedPixels"].strip()
                counter += 1
        

    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.name_frame[idx]+str(".png"))
        label = self.label_frame[idx]
        image = get_image(img_name, idx, self.size,self.Aug)
        width, height = image.size
        
        info = self.image_info[idx]
        if info["annotations"] != '-1':
            mask = rle2mask(info['annotations'], width, height)
            mask = Image.fromarray(mask.T)
            mask = np.expand_dims(mask, axis=0)
            masks = torch.as_tensor(mask, dtype=torch.float)
           
            
        else:
            mask = rle2mask('0 {}'.format(width*height), width, height)
            mask = Image.fromarray(mask.T)
            mask = np.expand_dims(mask, axis=0)
            masks = torch.as_tensor(mask, dtype=torch.float)
        
        if self.transform:
            image = self.transform(image);
            
                                
        
        return {'image': image,
                'labels': label,
                'Masks' : masks
                }
        

In [ ]:
IMAGE_TRAIN_DIR = "../input/siim-png-images/input/train_png/"
DF_PATH = "../input/trainsiimacr/train_siimacr/train-rle.csv"

In [ ]:
data_transf = transforms.Compose([transforms.ToTensor()])

In [ ]:
dataset = ImageDataLoader(train_file = image_ids ,image_dir = IMAGE_TRAIN_DIR  , size = 1024 , transform = data_transf)
item = dataset[2] # get some sample
image = item['image']
mask = item['Masks']


In [ ]:
mask = mask.numpy().squeeze()
image = transforms.ToPILImage()(image)
image = np.array(image)

In [ ]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(10, 10))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)

## Visualization 

In [ ]:
visualize(image, mask)

## HorizontalFlip

In [ ]:
def RandHorizontalFlip(image,mask = None,Prob = 1):
    aug = HorizontalFlip(p=Prob)
    augmented = aug(image=image, mask=mask)
    image = augmented['image']
    if mask is None : 
        return image 
    else : 
       
        mask = augmented['mask']
        return image,mask
        
image_Hflip,mask_Hflip =  RandHorizontalFlip(image,mask,Prob = 1)
visualize(image_Hflip, mask_Hflip, original_image=image, original_mask=mask)

## Vertical Flip

In [ ]:
def RandVerticalFlip(image,mask = None,Prob = 1):
    aug = VerticalFlip(p=Prob)
    augmented = aug(image=image, mask=mask)
    image = augmented['image']
    if mask is None : 
        return image 
    else : 
       
        mask = augmented['mask']
        return image,mask
        
image_Vflip,mask_Vflip =  RandVerticalFlip(image,mask,Prob = 1)
visualize(image_Vflip, mask_Vflip, original_image=image, original_mask=mask)


## Elastic Transform

In [ ]:

def RandElasticTransform(image,mask = None,Prob = 1):
    aug = ElasticTransform(p=1, alpha=150, sigma=120 * 0.05, alpha_affine=120 * 0.03)
    augmented = aug(image=image, mask=mask)
    image_elast = augmented['image']
    if mask is None : 
        return image_elast 
    else : 
       
        mask_elast = augmented['mask']
        return image_elast,mask_elast
        
image_elast,mask_elast =  RandElasticTransform(image,mask,Prob = 1)
visualize(image_elast, mask_elast, original_image=image, original_mask=mask)

## Grid Distortion

In [ ]:

def RandGridDistortion(image,mask = None,Prob = 1):
    aug = GridDistortion(p=Prob,distort_limit=0.35)
    
    augmented = aug(image=image, mask=mask)
    image_grid = augmented['image']
    if mask is None : 
        return image_grid 
    else : 
       
        mask_grid = augmented['mask']
        return image_grid,mask_grid
        
image_grid,mask_grid =  RandGridDistortion(image,mask,Prob = 1)
visualize(image_grid, mask_grid, original_image=image, original_mask=mask)

## RandomRotate90 

In [ ]:
def RandRotate90(image,mask = None,Prob = 1):
    aug = RandomRotate90(p=Prob)
    
    augmented = aug(image=image, mask=mask)
    image_grid = augmented['image']
    if mask is None : 
        return image_grid 
    else : 
       
        mask_grid = augmented['mask']
        return image_grid,mask_grid
        
image_RR,mask_RR =  RandRotate90(image,mask,Prob = 1)
visualize(image_RR, mask_RR, original_image=image, original_mask=mask)




## Model

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b4') 



in_features = model._fc.in_features


model.avg_pool = nn.AdaptiveAvgPool2d(1)
model._fc = nn.Sequential(
                          nn.BatchNorm1d(in_features, eps=0.001, momentum=0.01, affine=True, track_running_stats=True),
                          nn.Dropout(p=0.25),
                          nn.Linear(in_features=in_features, out_features=in_features, bias=True),
                          nn.ReLU(),
                          nn.BatchNorm1d(in_features, eps=0.001, momentum=0.01, affine=True, track_running_stats=True),
                          nn.Dropout(p=0.5),
                          nn.Linear(in_features=in_features, out_features=1, bias=True),
                         )
model = model.to(device)

## Train

Define the Loader again without the masks and with augmentation  

In [ ]:
def get_image(path, id_code, size,Aug):
    img_path = path
    
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    if Aug:
        image = RandGridDistortion(image,mask = None,Prob = 0.4)
        image = RandElasticTransform(image,mask = None,Prob = 0.4)
        image = RandRotate90(image,mask = None,Prob = 0.4)
        image = RandHorizontalFlip(image,mask = None,Prob = 0.4)
        
    image = cv2.resize(image, (size, size))
    image = transforms.ToPILImage()(image)
    return image

In [ ]:
class ImageDataLoaderClassifier(torch.utils.data.Dataset):

    def __init__(self,train_file,root_dir,size =256,transform=None,Aug = False):
        
        self.root_dir = root_dir
        self.size = size
        self.name_frame = train_file['ImageId'].values
        self.label_frame = train_file['Mask'].values
        self.transform = transform
        self.Aug = Aug

    def __len__(self):
        return len(self.name_frame)

    def __getitem__(self, idx):
        
       
       
        img_name = os.path.join(self.root_dir, self.name_frame[idx]+str(".png"))
        label = self.label_frame[idx]
        
        image = get_image(img_name, idx, self.size,self.Aug)
                 
              
        
        if self.transform:
            image = self.transform(image);
                                  
        
        return {'image': image,
                'labels': label
                }
          


In [ ]:
data_transf = transforms.Compose([transforms.ToTensor()])
batch_valid_size = 1
batch_size = 16
train_dataset = ImageDataLoaderClassifier(train_file = train  ,root_dir = IMAGE_TRAIN_DIR, size = 512 , transform = data_transf,Aug = True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_dataset = ImageDataLoaderClassifier(train_file = val  ,root_dir = IMAGE_TRAIN_DIR, size = 512 , transform = data_transf,Aug = False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_valid_size, shuffle=False, num_workers=4,drop_last =True)


In [ ]:
lr          = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
criterion = nn.BCEWithLogitsLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

In [ ]:
def train_model(epoch):
    model.train() 
        
    avg_loss = 0.
    optimizer.zero_grad()
    tk0 = tqdm_notebook(train_loader, total=int(len(train_loader)))
        
    for i, items in enumerate(tk0):
        
      imgs = items["image"]
      labels = items["labels"].view(-1, 1)
      imgs_train, labels_train = imgs.cuda(), labels.float().cuda()
      output_train = model(imgs_train)
      loss = criterion(output_train,labels_train)
       
      with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
         
      optimizer.step() 
      optimizer.zero_grad() 
      avg_loss += loss.item() / len(train_loader)
      tk0.set_postfix(loss= loss.item() / (batch_size))
      global_loss = loss
        
      
    return avg_loss





def test_model():
    
    avg_val_loss = 0.
    model.eval()
    with torch.no_grad():
      tk0 = tqdm_notebook(valid_loader, total=int(len(valid_loader)))
      for i, items in enumerate(tk0):
        imgs = items["image"]
        labels = items["labels"].view(-1, 1)
        imgs_vaild, labels_vaild = imgs.cuda(), labels.float().cuda()
        output_test = model(imgs_vaild)
        avg_val_loss += criterion(output_test, labels_vaild).item() / len(valid_loader)
        tk0.set_postfix(loss= criterion(output_test, labels_vaild).item() / len(valid_loader))       
        
    return avg_val_loss


In [ ]:
agg_train_loss = []
agg_val_loss = []

num_epochs = EPOCH
best_avg_loss = 100.0

global_loss = []



batch_loss = [] 

seed = SEED
seed_all(seed)
for epoch in range(num_epochs):
        
        
    avg_loss     = train_model(epoch)
    agg_train_loss.append(avg_loss)
    avg_val_loss = test_model()
    agg_val_loss.append(avg_val_loss)
    
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    print('lr:', scheduler.get_lr()[0]) 
       
    print('Training Loss: {:.4f}'.format(avg_loss))
    print('Validiation  Loss: {:.4f}'.format(avg_val_loss))
    
    scheduler.step()
    if avg_val_loss < best_avg_loss:
        best_avg_loss = avg_val_loss
        print("epoch is:",epoch,"nodel saved")
        torch.save(model.state_dict(), 'weight_bestf' + str(epoch) + '.pt')
        


## Plot the loss 

In [ ]:
figure(num=None, figsize=(15, 12), dpi=100, facecolor='w', edgecolor='k')
plt.plot(agg_train_loss)
plt.plot(agg_val_loss)
#plt.xticks(np.arange(0, len(batch_loss), batch_size)) 
plt.legend(['y = agg_train_loss', 'y = agg_val_loss'], loc='upper left')
plt.show()

## Predict 

In [ ]:
for param in model.parameters():
    param.requires_grad = False
model.eval()

In [ ]:
print((batch_size)) 
predict = np.zeros((batch_size)*len(valid_loader))
targets = np.zeros((batch_size)*len(valid_loader))
with torch.no_grad():
  tk0 = tqdm_notebook(valid_loader, total=int(len(valid_loader)))
  for i, items in enumerate(tk0):
    imgs = items["image"]
    labels = items["labels"].view(-1, 1)
    imgs_vaild, labels_vaild = imgs.cuda(), labels.float().cuda()
    output_test = model(imgs_vaild)
    predict[i*batch_size:(i+1)*batch_size] = output_test.cpu().detach().numpy().flatten()
    targets[i*batch_size:(i+1)*batch_size] = labels_vaild.cpu().detach().numpy().flatten()
    tk0.set_postfix(loss= criterion(output_test, labels_vaild).item() / len(valid_loader))       
        

In [ ]:
confusion_matrix(targets, predict> 0.5)